In [1]:
!pip install azure-storage-blob
#!pip install boto3
#!pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00


In [4]:
# import libaries

import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO


In [ ]:
s1 - gather - decompress - conver to tabular - clean - reformat - consolidation - transformation - load

In [5]:
# Azure function

def azure_upload_blob(connect_str, container_name, blob_name, data):
  blob_service_client = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
  blob_client.upload_blob(data, overwrite =True)
  print(f"Uploaded to Azure Blob: {blob_name} ")

def azure_download_blob(connect_str, container_name, blob_name):
  blob_service_client = BlobServiceClient.from_connection_string(connect_str)
  blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
  download_stream = blob_client.download_blob()
  return download_stream.readall()



In [6]:
URL = "https://data.cityofnewyork.us/resource/upii-frjc.csv?$limit=10000"

df_raw = pd.read_csv(URL, engine = 'pyarrow')
print(df_raw.info())
print(df_raw.shape)
df_raw.head()
print(df_raw.columns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   violation_number            10000 non-null  object             
 1   bic_number                  4701 non-null   float64            
 2   account_name                10000 non-null  object             
 3   type_of_violation           10000 non-null  object             
 4   violation_phase             10000 non-null  object             
 5   violation_disposition       10000 non-null  object             
 6   account_type                10000 non-null  object             
 7   violation_account_address   10000 non-null  object             
 8   violation_account_city      10000 non-null  object             
 9   violation_account_state     10000 non-null  object             
 10  violation_account_postcode  10000 non-null  object         

In [13]:
df_raw.dropna()
df_raw.shape

(10000, 31)

In [14]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   violation_number            10000 non-null  object             
 1   bic_number                  4701 non-null   float64            
 2   account_name                10000 non-null  object             
 3   type_of_violation           10000 non-null  object             
 4   violation_phase             10000 non-null  object             
 5   violation_disposition       10000 non-null  object             
 6   account_type                10000 non-null  object             
 7   violation_account_address   10000 non-null  object             
 8   violation_account_city      10000 non-null  object             
 9   violation_account_state     10000 non-null  object             
 10  violation_account_postcode  10000 non-null  object         

In [15]:
df_cleaned = df_raw.copy()

In [7]:
# specify the path to your Json configuration file
config_file_path = 'config.json'

# load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

#print the configuration
#Connection_STRING = (config["connectionString"])




CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]

CONTAINER_AZURE = "bic"
blob_name = "bic_20240503.csv"


# Convert Dataframe to CSV

output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

#Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite =True)

print(f"Uploaded {blob_name} to azure blob storage in container {CONTAINER_AZURE}. ")

Uploaded bic_20240503.csv to azure blob storage in container bic. 
